### **Script base: Spazzata di voltaggio**

Questo script esegue una spazzata lineare dal voltaggio `V0` al voltaggio `V1` con  `nV` punti. Per ogni valore l'$\texttt{ADC}$ esegue una misura con i seguenti parametri di campionamento:

* frequenza di sampling `fs`
* numero di acquisizioni `npt`

I valori risultanti di $\texttt{Ch1}$ e $\texttt{Ch2}$ vengono mediati e salvati.

In [ ]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import time
import os

#PRIMA DI RUNNARE VAI IN FONDO LA CODICE ED INSERISCI TUTTI I DATI MANCANTI.


#Dal task 2 so che voltaggio scegliere, dovrà essere V0 = V1.

V0 = 2.1
V1 = 2.1
nV = 50
npt = 50
fs  = 1e6
R = 1e4  #INSERIRE IL VALORE DELLA RESISTENZA USATA

Vv = np.linspace(V0,V1,nV)

#ATTENZIONE: Il codice è pensato in modo che si prenda prima una misura di I0, poi di Id, quindi ogni volta che cambio concentrazione, prima di prendere la nuova cuvetta, misuro I0 con la cuvetta piena d'acqua.

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione 
ad2 = tdwf.AD2()
ad2.vdd = 5
ad2.vss = -5
ad2.power(True)
#   3. Configurazione generatore funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.func = tdwf.funcDC
#wgen.w1.offs = V0
wgen.w1.start()
#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=fs
scope.npt=npt
scope.ch2.avg = True
scope.ch1.rng = 50
scope.ch2.rng = 50

# -[Ciclo di misura]-----------------------------------------------------------
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 6))
fig.canvas.manager.set_window_title('Spazzata voltaggio')
ax2.yaxis.tick_right()
Ch1v = np.full(nV, np.nan)
Ch2v = np.full(nV, np.nan)
s_Ch2v = np.full(nV, np.nan)
for ii, V in enumerate(Vv):
    wgen.w1.offs = V
    time.sleep(0.1)
    scope.sample()
    Ch1v[ii] = np.mean(scope.ch1.vals)
    Ch2v[ii] = np.mean(scope.ch2.vals)
    s_Ch2v[ii] = np.std(scope.ch2.vals)

    ax1.clear()    
    ax1.plot(Vv, Ch1v, ".", color="tab:orange", label="Ch1")
    ax1.plot(Vv, Ch2v, ".", color="tab:blue", label="Ch2")
    ax1.grid(True)
    ax1.set_xlabel("W1 [V]", fontsize=15)
    ax1.set_ylabel("Signals [V]", fontsize=15)
    ax1.legend()
    
    ax2.clear()    
    ax2.plot(Ch1v, Ch2v, ".", color="tab:orange")
    ax2.grid(True)
    ax2.set_xlabel("Ch1 [V]", fontsize=15)
    ax2.set_ylabel("Ch2 [V]", fontsize=15)
    ax2.yaxis.set_label_position('right')
    plt.tight_layout()
    plt.show(block=False)
    plt.pause(0.001)

ad2.close()

path = "/home/marco/Desktop/Uni_anno3/TD/Es_07/acquisizioni/"
name = "I_d_LED_blu.txt"

file_exists = os.path.isfile(path+name)

with open(path+name, "a") as f:
    if not file_exists:
        f.write("#Id\ts_Id\n")
    f.write(f"{np.mean(Ch2v)/R}\t{np.std(Ch2v/R)/np.sqrt(nV)}\n")



'\npath = "/home/marco/Desktop/Uni_anno3/TD/Es_07/acquisizioni/"\nname = "I_d_LED_rosso.txt"  #ricorda di cambiare il colore del LED nel nome del file\n\nfile_exists = os.path.isfile(path+name)\n\nwith open(path+name, "a") as f:\n\n    if not file_exists:\n        f.write("#Id\ts_Id\n")\n\n    f.write(f"{np.mean(Ch2v)/R}\t{np.mean(s_Ch2v)/(R*np.sqrt(nV))}\n")\n\nVW = []\nVL = []\ns_VW = []\ns_VL = []\n\nVW = np.array(VW)\nVL = np.array(VL)\ns_VW = np.array(s_VW)\ns_VL = np.array(s_VL)\n\n# number of points\nx = len(VW)     #x = NUMERO DI CAMPIONI CON CONCENTRAZIONI DIVERSE SCELTI, DA INSERIRE PRIMA DI RUNNARE IL CODICE\n\nconcentrazioni = np.zeros((x, 2))\n\nconcentrazioni[:,0] = VL/(VW + VL)\n\nconcentrazioni[:,1] = (\n    1/(VW+VL)**2\n    * np.sqrt((VL*s_VW)**2 + (VW*s_VL)**2)\n)\n\nsave_conc = False\n\nif save_conc:\n    np.savetxt(path+"concentrazioni.txt", concentrazioni, delimiter=\'\t\', \n           header="#Conc[V]\t#s_Conc[V]")\n\n'

In [ ]:
VW = np.array([4.0, 3.0, 2.0, 1.0, 0.0])   # fill before running    in ml
VL = np.array([0.0, 1.0, 2.0, 3.0, 4.0])   # fill before running

s_VW = np.full(VW.shape, .5)
s_VL = np.full(VW.shape, .5)

#s_VW = np.array([0.5]) # fill before running
#s_VL = np.array([0.5]) # fill before running

x = len(VW)

if x == 0:
    print("ERROR: VW/VL arrays are empty — insert your data before running!")
    exit()

concentrazioni = np.zeros((x, 2))

concentrazioni[:,0] = VL / (VW + VL)

concentrazioni[:,1] = (
    1/(VW+VL)**2 * np.sqrt((VL*s_VW)**2 + (VW*s_VL)**2)
)

save_conc = False

if save_conc:
    path = "/home/marco/Desktop/Uni_anno3/TD/Es_07/acquisizioni/"
    np.savetxt(path+"concentrazioni.txt", concentrazioni, delimiter='\t',
               header="#Conc[V]\t#s_Conc[V]")

